This is a jupyter file that has been put together for the datasciencepro capstone course in place of using the online plotly tool

In [1]:
# install plotly
!pip install plotly
!pip install pandas

In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import requests
import os

In [3]:
from urllib.request import urlretrieve

url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"
urlretrieve(url, "spacex_launch_dash.csv")
    
    

('spacex_launch_dash.csv', <http.client.HTTPMessage at 0x239628ecd90>)

In [4]:
df = pd.read_csv("spacex_launch_dash.csv")
print(df.head())

   Unnamed: 0  Flight Number  Launch Site  class  Payload Mass (kg)  \
0           0              1  CCAFS LC-40      0                0.0   
1           1              2  CCAFS LC-40      0                0.0   
2           2              3  CCAFS LC-40      0              525.0   
3           3              4  CCAFS LC-40      0              500.0   
4           4              5  CCAFS LC-40      0              677.0   

  Booster Version Booster Version Category  
0  F9 v1.0  B0003                     v1.0  
1  F9 v1.0  B0004                     v1.0  
2  F9 v1.0  B0005                     v1.0  
3  F9 v1.0  B0006                     v1.0  
4  F9 v1.0  B0007                     v1.0  


In [5]:
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py"
urlretrieve(url, "spacex_dash_app.py")

('spacex_dash_app.py', <http.client.HTTPMessage at 0x23904805dd0>)

In [6]:
from dash import html, dcc

In [7]:
import platform
import subprocess

def stop_dash_app():
    system = platform.system()
    if system == "Windows":
        try:
            result = subprocess.check_output("netstat -ano | findstr :8050", shell=True).decode()
            pid = result.split()[-1]
            subprocess.check_output(f"taskkill /PID {pid} /F", shell=True)
            print(f"Process {pid} has been terminated.")
        except subprocess.CalledProcessError:
            print("No process found using port 8050.")
    elif system in ["Darwin", "Linux"]:  # macOS or Linux
        try:
            result = subprocess.check_output("lsof -ti :8050", shell=True).decode()
            pid = result.strip()
            subprocess.check_output(f"kill -9 {pid}", shell=True)
            print(f"Process {pid} has been terminated.")
        except subprocess.CalledProcessError:
            print("No process found using port 8050.")
    else:
        print(f"Unsupported operating system: {system}")

stop_dash_app()

Process 3528 has been terminated.


In [10]:
# %load spacex_dash_app.py
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                            ],
                                            value='ALL',
                                            placeholder="Select a Launch Site here",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                                                value=[min_payload, max_payload]),
                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values='class', 
        names='Launch Site', 
        title='Total Success Launches By Site')
        return fig
    else:
        # return the outcomes piechart for a selected site
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name='class count')
        fig = px.pie(filtered_df, values='class count', 
        names='class',
        title=f'Total Success Launched for site {entered_site}')
        return fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), 
              Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site, payload):
    low, high = payload
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] > low) & (spacex_df['Payload Mass (kg)'] < high)]       
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x='Payload Mass (kg)', y='class', 
        color='Booster Version Category',
        title='Correlation between Payload and Success for all Sites')
        return fig
    else:
        fig = px.scatter(filtered_df[filtered_df['Launch Site'] == entered_site],
        x='Payload Mass (kg)', y='class',
        color='Booster Version Category',
        title=f'Correlation between Payload and Success for site {entered_site}')
        return fig

# Run the app
if __name__ == '__main__':
    app.run_server()
